In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter

from survival_utils import CoxPH_train_val, CoxPH_train_infer, clinic_eval, extract_train_val_hazard_ratios

In [2]:
plots_save_path = 'all_feats_combi/KM_plots/'
##path to the csv file containing features for all the sets/cases including train/val
discov_val_feats_path = 'all_feats_combi/features_combined/NOTT/discovery_valid_combined.csv'
discov_df = pd.read_csv(discov_val_feats_path)
all_feats_list = list(discov_df.columns[39:]) # start from digital features in the matrix

# clear the features based on std
feats_list = []
features_std = discov_df.std()
for feat in all_feats_list:
    std_value = features_std[feat]
    if std_value > 0.5:
       feats_list.append(feat) 

# defining the parameters for survival analysis
model_type = 'cox' ## 'rsf': for Random Survival Forest, 'cox': for Cox PH regression model
save_plot = True ##whether to save the KM curve plots
censor_at = 120 #in months. e.g 10 years = 120 months. 180 for 15 years, 240 for 20 years. Use -1 if no censoring is required 
rsf_rseed = 100 ## random seed for Random Survival Forest
cutoff_mode = 'median' ## 'median' | 'mean' ## the cut off point calculation for stratification of high vs low risk cases
cutoff_point =  -1 ## 0.92 ##if set to -1 then median will be used as cut off. If set to any other positive value then cut_mode option will be ignores and the fixed cut off provided will be used for stratification of high vs low risk cases
time_col = 'TTDM/ month' #'TTDM/ month' | 'Breast cancer specific survival/ month'
event_col = 'Distant Metastasis' #'Distant Metastasis' | 'Survival Status'
subset = 'Endocrine_LN0' ##'Endocrine'|'Endocrine_LN0'; 'Endocrine': Endocrine treated only with lymph node 0-3; 'Endocrine_LN0': Endocrine treated lymph node negative

######################################### Start the bootstraping
# initialize the output lists
bootsrap_num = 10
hr_scores = pd.DataFrame(1, index=np.arange(bootsrap_num), columns=feats_list)
EE = discov_df[event_col].to_numpy()
rng = np.random.RandomState()
for run in tqdm(range(1)):
    index_train = list(rng.choice(np.nonzero(EE==0)[0],size = len(EE)-np.sum(EE),replace=True))+list(rng.choice(np.nonzero(EE==1)[0],size = np.sum(EE),replace=True))
    index_test = list(set(range(len(EE))).difference(index_train))
    
    train_set = discov_df.iloc[index_train]
    test_set = discov_df.iloc[index_test]
    
    train_set.reset_index(inplace=True)
    test_set.reset_index(inplace=True)
    # tpv, tcind, thz, thz_ci_l, thz_ci_h, vpv, vcind, vhz, vhz_ci_l, vhz_ci_h, cutpnt, hr_scores = CoxPH_train_val(train_set, test_set, plots_save_path, feats_list, time_col, event_col, subset, censor_at, save_plot, cutoff_mode, cutoff_point, hr_scores, 0)
    
    if run == 0:
        test_hazard_ratios, c_index, p_value = extract_train_val_hazard_ratios(train_set, test_set, plots_save_path, feats_list, time_col, event_col, subset, censor_at, save_plot, cutoff_mode, cutoff_point)
        
        # print(test_hazard_ratios.tolist())
        # print(len(test_hazard_ratios.tolist()))

/tmp/ipykernel_18391/919052237.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  features_std = discov_df.std()
100%|██████████| 1/1 [00:25<00:00, 25.78s/it]


In [6]:
test_hazard_ratios = pd.concat([test_hazard_ratios, test_hazard_ratios], axis=1, join='outer', ignore_index=True, sort=False)

In [7]:
features_std['nodeDegrees_max']

2.9712343863942743

In [14]:
hrs_mean = test_hazard_ratios.mean(axis=1)
hrs_mean

covariate
g1_g3_ratio_g2                                 1.004749
g1_ratio_g2                                    1.000157
g1_ratio_g3                                    0.994123
g2_g3_ratio_g1                                 0.857903
g2_ratio_g3                                    1.000000
                                                 ...   
ratio_intra_WSI_p1p3_patch_cooccur_low_high    1.006479
ratio_intra_WSI_p2p2_patch_cooccur_low_high    0.988744
ratio_intra_WSI_p2p3_patch_cooccur_low_high    0.986664
ratio_intra_WSI_p3p3_patch_cooccur_low_high    1.027865
stromal_contrast                               0.863004
Length: 265, dtype: float64

In [18]:
def normalize_df(train_set, test_set, feats_list):
    feat_mean = train_set[feats_list].mean()
    feat_std = train_set[feats_list].std()
    
    train_set_normalized = train_set.copy()
    test_set_normalized = test_set.copy()
    train_set_normalized[feats_list] =( train_set[feats_list] - feat_mean ) / feat_std
    test_set_normalized[feats_list] =( test_set[feats_list] - feat_mean ) / feat_std
    
    return train_set_normalized, test_set_normalized

train_set_n1, test_set_n1 = normalize_df(train_set, test_set, feats_list)

In [23]:
def normalize_df_2(train_set, test_set, feats_list):
    for feat in feats_list:
        feat_mean = train_set[feat].mean()
        feat_std = train_set[feat].std()
    
        train_set[feat] =( train_set[feat] - feat_mean ) / feat_std
        test_set[feat] =( test_set[feat] - feat_mean ) / feat_std
    
    return train_set, test_set

train_set_n2, test_set_n2 = normalize_df_2(train_set, test_set, feats_list)

/tmp/ipykernel_12047/2133161088.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set[feat] =( test_set[feat] - feat_mean ) / feat_std


In [25]:
print(train_set_n2.equals(train_set_n1)) 

False
